<center>

<font size="5">MGMTMSA 403: Optimization</font>

</center>


<center>

<font size="5">Assignment 2: Portfolio Optimization</font>

</center>


<div style="display: flex; justify-content: space-between;">
    <div style="text-align: left;">
        Due on BruinLearn by 11:59pm on January 31st.
    </div>
    <div style="text-align: right;">
        Arnav Garg (906310841) <br>
        Oscar Trumpy (706310371) <br>
        Facundo Klappenbach (306316186) <br>
    </div>
</div>


**<font size="5">Background</font>**

The file Prices.csv contains monthly prices for 390 stocks, collected over a 5-year period. Each column of the file corresponds to one stock. The first row of the file contains the ticker of each stock, which is a 3-4 letter that identifies the company. For example, the ticker for Microsoft is ’MSFT’.
<br>
<br>
This assignment will require you to formulate and solve three different portfolio optimization models. In order to formulate the models, you will first have to compute the monthly returns for each stock, as well as the covariance matrix. Let t = 1, . . . , 60 index the months over the 5-year period, and let Price_t be the price of a stock in period t. The average monthly return of the stock (in %) in period t can then be calculated as

\begin{align}
{Return_t} = \frac{Price_{t+1} - Price_t}{Price_t} \times 100\%
\end{align}

Note that the covariance of a matrix M can be calculated using the function numpy.cov(M) from the NumPy package.
<br>
<br>
**HINT:** You may use the Assignment 2 Preprocessing file posted on BruinLearn to compute the average returns and covariance information.

In [1]:
##############################################################################################
# Data Pre-Processing
##############################################################################################
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter = ',', skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d - 1, n))
for stock in range(n):
    for month in range(d - 1):
        returns[month, stock] = prices[month + 1, stock] / prices[month, stock] - 1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns, axis = 0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n, n))
C = np.cov(np.transpose(returns))
##############################################################################################


**<font size="5">Models</font>**

The description of each model is given below.
<br>
<br>
**Model 1.** Start by focusing on a four-asset portfolio: Suppose you can only invest in Microsoft (MSFT), Goldman Sachs (GS), Proctor & Gamble (PG), and U.S. Treasury Bonds (SCHP). Construct a minimum-variance portfolio with an expected monthly return of at least 0.5%.

\begin{align}
\text{minimize} \;\; & \sum_{i=1}^4 \sum_{j=1}^4 w_i \times w_j \times C_{ij} \\
\text{s.t.} \;\; & \sum_{i=1}^4 w_i \times r_i \ge 0.5 \\
            & \sum_{i=1}^4 w_i = 1 \\
            & w_i \ge 0 \quad i = 1 \ldots 4 \\
\end{align}

In [2]:
##############################################################################################
# Initialize the Model 1
mod1 = Model()

##############################################################################################
# Define Variables 
# Defining the weights variable
w1 = mod1.addVars(len(tickers))

##############################################################################################
# Define Constraints
## Expected return should atleast be 0.5
mod1.addConstr(sum(w1[i] *  avg_return[i] for i in tickind) >= 0.005)

## Weights add up to 1
mod1.addConstr(sum(w1[i] for i in tickind) == 1)

## w_i >= 0
for i in tickind:
    mod1.addConstr(w1[i] >= 0)

##############################################################################################
# Construct Objective
mod1.setObjective(sum((w1[i] * w1[j] * C[i, j]) for i in tickind for j in tickind), GRB.MINIMIZE)

##############################################################################################
# Update and solve
mod1.update()
mod1.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Value:", mod1.objVal)

##############################################################################################

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 390 columns and 12 nonzeros
Model fingerprint: 0x24944e33
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 386 columns
Presolve time: 0.04s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal    

**Model 2.** Now suppose you can invest in all 390 stocks. Construct a minimum-variance portfolio with an expected monthly return of at least 0.5%.
\begin{align}
\text{minimize} \;\; & \sum_{i=1}^{390} \sum_{j=1}^{390} w_i \times w_j \times C_{ij} \\
\text{s.t.} \;\; & \sum_{i=1}^{390} w_i \times r_i \ge 0.5 \\
            & \sum_{i=1}^{390} w_i = 1 \\
            & w_i \ge 0 \quad i = 1 \ldots 390 \\
\end{align}


In [3]:
##############################################################################################
# Initialize the Model 2
mod2 = Model()

##############################################################################################
# Define Variables 
# Defining the weights variable
w2 = mod2.addVars(len(tickers))

##############################################################################################
# Define Constraints
## Expected return should atleast be 0.5
mod2.addConstr(sum(w2[i] *  avg_return[i] for i in range(len(tickers))) >= 0.005)

## Weights add up to 1
mod2.addConstr(sum(w2[i] for i in range(len(tickers))) == 1)

## w_i >= 0
for i in range(len(tickers)):
    mod2.addConstr(w2[i] >= 0)

##############################################################################################
# Construct Objective
mod2.setObjective(sum((w2[i] * w2[j] * C[i, j]) for i in range(len(tickers)) for j in range(len(tickers))), GRB.MINIMIZE)

##############################################################################################
# Update and solve
mod2.update()
mod2.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Value:", mod2.objVal)

##############################################################################################

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0xd3a53ccf
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633e-17 -

**Model 3.** In practice, there are transaction fees associated with buying stocks. One way of keeping transaction fees low while still attaining desirable performance is to limit the total number of stocks that are purchased (i.e. limit the number of stocks that have a strictly positive weight). Construct a minimum-variance portfolio that selects at most 4 of the 390 stocks, and has an expected monthly return of at least 0.5%. (Note: By introducing binary variables into a quadratic program, we obtain a **quadratic integer program.** Fortunately, this particular quadratic integer program can be solved by Gurobi.)
\begin{align}
\text{minimize} \;\; & \sum_{i=1}^{390} \sum_{j=1}^{390} w_i \times w_j \times C_{ij} \\
\text{s.t.} \;\; & X_i \in \{0, 1 \} \quad i = 1 \ldots 390 \\
            & \sum_{i=1}^{390} X_i <= 4 \\
            & \sum_{i=1}^{390} w_i \times r_i \ge 0.5 \\
            & \sum_{i=1}^{390} w_i = 1 \\
            & w_i <= X_i \quad i = 1 \ldots 390 \\
            & w_i \ge 0 \quad i = 1 \ldots 390 \\
\end{align}

In [4]:
##############################################################################################
# Initialize the Model 3
mod3 = Model()

##############################################################################################
# Define Variables 
# Defining the weights variable
w3 = mod3.addVars(len(tickers))
# Dfeining the binary variable for choice
x = mod3.addVars(len(tickers), vtype = GRB.BINARY)

##############################################################################################
# Define Constraints
## Expected return should atleast be 0.5
mod3.addConstr(sum(w3[i] *  avg_return[i] for i in range(len(tickers))) >= 0.005)

## Weights add up to 1
mod3.addConstr(sum(w3[i] for i in range(len(tickers))) == 1)

## w_i >= 0
for i in range(len(tickers)):
    mod3.addConstr(w3[i] >= 0)

## Sum of X_i <= 4
mod3.addConstr(sum(x[i] for i in range(len(tickers))) <= 4)

## Weights should be lessthan binary variable
for i in range(len(tickers)):
    mod3.addConstr(w3[i] <= x[i])

##############################################################################################
# Construct Objective
mod3.setObjective(sum((w3[i] * w3[j] * C[i, j]) for i in range(len(tickers)) for j in range(len(tickers))), GRB.MINIMIZE)

##############################################################################################
# Update and solve
mod3.update()
mod3.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Value:", mod3.objVal)

##############################################################################################

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x6a25f44d
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

**<font size="5">Questions</font>**

**1.** Formulate and solve each of the three models in Python, and then answer the following questions. For each part, also include your Gurobi output.


a) For **Model 1**, write down the optimal risk (i.e. the optimal objective function value), solver time, and the weight on each of the four stocks.

In [5]:
print("Optimal Value:", round(mod1.objVal, 6))

print("Solver Time:", round(mod1.Runtime, 6), "s")

for i in tickind:
    print("Weight of index", i, ":", round(w1[i].x, 6))

Optimal Value: 0.000177
Solver Time: 0.053893 s
Weight of index 315 : 0.237118
Weight of index 216 : 0.02586
Weight of index 372 : 0.0
Weight of index 388 : 0.737023


b) For **Model 2**, write down the optimal risk and solver time.


In [6]:
print("Optimal Value:", round(mod2.objVal, 6))

print("Solver Time:", round(mod2.Runtime, 6), "s")

Optimal Value: 2.9e-05
Solver Time: 0.042813 s


c) For **Model 3**, report the optimal risk, solver time, and the ticker and weight on each of the four stocks selected by the model.

In [7]:
print("Optimal Value:", round(mod3.objVal, 6))

print("Solver Time:", round(mod3.Runtime, 6), "s")

for i in range(len(tickers)):
    if x[i].x == 1:
        print("Weight of index", i, ":", round(w3[i].x, 6))

Optimal Value: 6.8e-05
Solver Time: 16.162996 s
Weight of index 118 : 0.126411
Weight of index 285 : 0.075476
Weight of index 348 : 0.043754
Weight of index 389 : 0.754359


**2.** Use your solution to Question 1 above to answer the following questions:
<br>
<br>
a) Is the optimal risk in **Model 1** higher or lower than **Model 2**? Explain why in 1-2 sentences.
<br>
The optimal risk in Model 2 is lower than the optimal risk in Model 1 because in Model 2 we are considering all 390 assets to create an optimal portfolio. In Model 1, we are limiting the scope of our optimisation problem to only 4 assets. Therefore, Model 2 is able to choose a better combination of assets which can give a lower optimal risk for the same return.
<br>
<br>
b) Is the optimal risk in **Model 2** higher or lower than **Model 3**? Explain why in 1-2 sentences.
<br>
The optimal risk in Model 3 is higher than the optimal risk in Model 2. Even though Model 3 finds the best combination of 4 assets with minimum variance, the scope of its optimisation is limited as compared to Model 2 because Model 2 is able to choose the best combination of assets out of all 390 assets. Therefore, Model 2 is able to choose a better combination of assets which can give a lower optimal risk for the same return.

**3.**  In some cases, we may want to get an approximate solution quickly by terminating the branch- and-bound algorithm before it finds an optimal solution. There are two ways to terminate Gurobi early: (a) by setting a maximum time limit, and (b) by setting a maximum acceptable optimality gap (the tolerance). Use **Model 3** to answer the following two questions. For each part, also include your Gurobi output.


a) Set Gurobi to terminate after 3 seconds by including XYZ.Params.TimeLimit = 3.0 in your code for **Model 3**, where ’XYZ’ is the name of your model. How does the objective function value at termination compare the optimal value obtained in question 1c)?
<br>
<br>
The objective function value at termination is exactly equal to the value obtained in question 1c).

In [8]:
##############################################################################################
# Initialize the Model 3
mod3 = Model()
mod3.Params.TimeLimit = 3.0
##############################################################################################
# Define Variables 
# Defining the weights variable
w3 = mod3.addVars(len(tickers))
# Dfeining the binary variable for choice
x = mod3.addVars(len(tickers), vtype = GRB.BINARY)

##############################################################################################
# Define Constraints
## Expected return should atleast be 0.5
mod3.addConstr(sum(w3[i] *  avg_return[i] for i in range(len(tickers))) >= 0.005)

## Weights add up to 1
mod3.addConstr(sum(w3[i] for i in range(len(tickers))) == 1)

## w_i >= 0
for i in range(len(tickers)):
    mod3.addConstr(w3[i] >= 0)

## Sum of X_i <= 4
mod3.addConstr(sum(x[i] for i in range(len(tickers))) <= 4)

## Weights should be lessthan binary variable
for i in range(len(tickers)):
    mod3.addConstr(w3[i] <= x[i])

##############################################################################################
# Construct Objective
mod3.setObjective(sum((w3[i] * w3[j] * C[i, j]) for i in range(len(tickers)) for j in range(len(tickers))), GRB.MINIMIZE)

##############################################################################################
# Update and solve
mod3.update()
mod3.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Value:", mod3.objVal)

##############################################################################################

Set parameter TimeLimit to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x6a25f44d
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Obje

b) Set Gurobi to terminate after reaching a gap of 10% by including XYZ.Params.MIPGap = 0.1 in your code for **Model 3**, where ’XYZ’ is the name of your model. (Note: The default gap in Gurobi is 0.0001.) How does the solver time compare with the solution time obtained in question 1c)?
<br>
<br>
The solver time after reaching a gap of 10% is shorter than the solver time obtained in question 1c).

In [9]:
##############################################################################################
# Initialize the Model 3
mod3 = Model()
mod3.Params.MIPGap = 0.1
##############################################################################################
# Define Variables 
# Defining the weights variable
w3 = mod3.addVars(len(tickers))
# Dfeining the binary variable for choice
x = mod3.addVars(len(tickers), vtype = GRB.BINARY)

##############################################################################################
# Define Constraints
## Expected return should atleast be 0.5
mod3.addConstr(sum(w3[i] *  avg_return[i] for i in range(len(tickers))) >= 0.005)

## Weights add up to 1
mod3.addConstr(sum(w3[i] for i in range(len(tickers))) == 1)

## w_i >= 0
for i in range(len(tickers)):
    mod3.addConstr(w3[i] >= 0)

## Sum of X_i <= 4
mod3.addConstr(sum(x[i] for i in range(len(tickers))) <= 4)

## Weights should be lessthan binary variable
for i in range(len(tickers)):
    mod3.addConstr(w3[i] <= x[i])

##############################################################################################
# Construct Objective
mod3.setObjective(sum((w3[i] * w3[j] * C[i, j]) for i in range(len(tickers)) for j in range(len(tickers))), GRB.MINIMIZE)

##############################################################################################
# Update and solve
mod3.update()
mod3.optimize()

##############################################################################################
# Print Optimal Solution
print("Solver Time:", mod3.Runtime)

##############################################################################################

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x6a25f44d
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objec